In [1]:
import pulp

In [2]:
# coefficients nominal values
a = [[1,2],[3,4]]

In [3]:
# a_hats in normalised deviations from nominal values, s.t. a_i belongs to [a_bar - a_hat, a_bar + a_hat]
a_hat = [[1,0.5],[0.8,1.5]]

In [4]:
# rhs
b=[6,6]

In [5]:
# Gamma_i represents the budget of uncertainty for each constraint row, such that it is the upper bound 
# on the sum of absolute values of normalised deviations d_ij
Gamma = [0.0,0.0]

In [7]:
# Create maximization model
testproblem = pulp.LpProblem('Testproblem', pulp.LpMaximize)

In [8]:
# Decision variables
x = {}
for j in range(2):
    x[j] = pulp.LpVariable('x[%s]' % j, 0)

In [9]:
# Define y_j >= 0, these are the bounds on x
y = {}
for j in range(2):
    y[j] = pulp.LpVariable('y[%s]' % j, 0)

In [10]:
# Define z_i >= 0, from Bertsimas & Sim Theorem 1 equation 7
z = {}
for i in range(2):
    z[i] = pulp.LpVariable('z[%s]' % i, 0)

In [11]:
# Define p_ij >= 0, from Bertsimas & Sim Theorem 1 equation 7
p = {}
for i in range(2):
    for j in range(2):
        p[i, j] = pulp.LpVariable('p[%s,%s]' % (i,j), 0)

In [12]:
# Set objective as simply x1+x2
objective = pulp.LpVariable('Objective', 0)
testproblem += objective == sum(x[j] for j in range(2))
testproblem.setObjective(objective)

In [13]:
# Set constraints
for i in range(2):
    testproblem += sum(a[i][j] * x[j] for j in range(2)) + Gamma[i] * z[i] + sum(p[i, j] for j in range(2)) <= b[i]

In [14]:
# Set deviations for uncertain parameters
for i in range(2):
    for j in range(2):
        testproblem += z[i] + p[i, j] >= a_hat[i][j] * y[j]

In [15]:
# Relate deviations to the decision variables
for j in range(2):
    testproblem += x[j] <= y[j]

In [16]:
testproblem

Testproblem:
MAXIMIZE
1*Objective + 0.0
SUBJECT TO
_C1: Objective - x_0_ - x_1_ = 0

_C2: p_0,0_ + p_0,1_ + x_0_ + 2 x_1_ <= 6

_C3: p_1,0_ + p_1,1_ + 3 x_0_ + 4 x_1_ <= 6

_C4: p_0,0_ - y_0_ + z_0_ >= 0

_C5: p_0,1_ - 0.5 y_1_ + z_0_ >= 0

_C6: p_1,0_ - 0.8 y_0_ + z_1_ >= 0

_C7: p_1,1_ - 1.5 y_1_ + z_1_ >= 0

_C8: x_0_ - y_0_ <= 0

_C9: x_1_ - y_1_ <= 0

VARIABLES
Objective Continuous
p_0,0_ Continuous
p_0,1_ Continuous
p_1,0_ Continuous
p_1,1_ Continuous
x_0_ Continuous
x_1_ Continuous
y_0_ Continuous
y_1_ Continuous
z_0_ Continuous
z_1_ Continuous

In [17]:
# Solve the model
status = testproblem.solve()
assert status == pulp.LpStatusOptimal

In [20]:
pulp.value(objective)

2.0